In [5]:
import requests
from fp.fp import FreeProxy

def calling_api(lat_long):
    payload = {"tab_id": "home_tab",
               "ne_lat" : lat_long[0],
               "ne_lng" : lat_long[1],
               "sw_lat" : lat_long[2],
               "sw_lng" : lat_long[3]}
    
    proxy = FreeProxy(rand=True).get()

    header_val ={"x-airbnb-api-key":"d306zoyjsyarp7ifhu67rjxn52tv0t20"}
    r = requests.get("https://api.airbnb.com/v2/explore_tabs", headers = header_val, params=payload, proxies = {"http":proxy})
    return r 

In [6]:
def square_corners(coord):
    ne_lat,ne_lng,sw_lat,sw_lng = coord[0], coord[1], coord[2], coord[3]
    lng_dif = ne_lng - sw_lng
    mid_lng = lng_dif/2 + sw_lng
    lat_dif = ne_lat - sw_lat
    mid_lat = lat_dif/2 + sw_lat   
    
    sq1 = [ne_lat,mid_lng,mid_lat,sw_lng]
    sq2 = [mid_lat,mid_lng,sw_lat,sw_lng]
    sq3 = [ne_lat,ne_lng,mid_lat,mid_lng]
    sq4 = [mid_lat,ne_lng,sw_lat,mid_lng]
    
    return sq1,sq2,sq3,sq4

In [7]:
listing_ids_list = dict()

In [8]:
import queue
import time

q = queue.Queue()
q.put([42.396796, -70.921061, 42.227402, -71.190610])

while q.empty() is not True:
    try:
        coord = q.get()
        result_api = calling_api(coord)
        result_api_json = result_api.json()

        traverse_json = result_api_json['explore_tabs'][0]
        for i in traverse_json["sections"]:
                if i["result_type"] == "listings":
                    for k in i["listings"]:
                        if k["listing"]["id"] in listing_ids_list:
                            continue
                        else:
                            listing_ids_list[k["listing"]["id"]] = 1

        if traverse_json["home_tab_metadata"]['listings_count'] > 1:
            sq1,sq2,sq3,sq4 = square_corners(coord)
            q.put(sq1)
            q.put(sq2)
            q.put(sq3)
            q.put(sq4)

        print(q.qsize(),len(listing_ids_list),traverse_json["home_tab_metadata"]['listings_count'])
        
    except:
        q.put(coord)
        pass
        
 

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
listing_ids_list